<a href="https://colab.research.google.com/github/gcunharodrigues/fcc_nn_sms_text_classifier/blob/main/fcc_nn_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
  !pip install nlp
  !pip install nltk
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import re
import nlp
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

print(tf.__version__)

ERROR: unknown command "instal" - maybe you meant "install"
2.19.0-dev20250129


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import string

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-01-30 14:49:53--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.02s   

2025-01-30 14:49:53 (16.2 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-01-30 14:49:53--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
# import data
train_df = pd.read_csv(train_file_path, sep='\t', names=['predict', 'text'])
val_df = pd.read_csv(test_file_path, sep='\t', names=['predict', 'text'])

In [ ]:
# Check missing or null values
train_df.isna().sum()

,0
predict,0
text,0


In [ ]:
val_df.isna().sum()

,0
predict,0
text,0


In [ ]:
# Checking duplicates
train_df.duplicated().sum()

244

In [ ]:
train_df[train_df.duplicated(keep=False)].sort_values(by=['text'])

,predict,text
996,ham,1) go to write msg 2) put on dictionary mode 3...
3089,ham,1) go to write msg 2) put on dictionary mode 3...
3707,ham,1) go to write msg 2) put on dictionary mode 3...
953,spam,18 days to euro2004 kickoff! u will be kept in...
523,spam,18 days to euro2004 kickoff! u will be kept in...
...,...,...
4041,ham,your opinion about me? 1. over 2. jada 3. kusr...
1749,ham,yup
1986,ham,yup
2894,ham,yup ok...


In [ ]:
val_df[val_df.duplicated(keep=False)].sort_values(by=['text'])

,predict,text
60,spam,december only! had your mobile 11mths+? you ar...
823,spam,december only! had your mobile 11mths+? you ar...
486,ham,did u got that persons story
13,ham,did u got that persons story
64,ham,"dont search love, let love find u. thats why i..."
...,...,...
1040,spam,you have 1 new voicemail. please call 08719181...
761,spam,you have won a nokia 7250i. this is what you g...
286,spam,you have won a nokia 7250i. this is what you g...
1121,ham,your opinion about me? 1. over 2. jada 3. kusr...


In [ ]:
# Clean dataframe
train_df = train_df.drop_duplicates()
val_df = val_df.drop_duplicates()

In [ ]:
train1_df = train_df.copy()
val1_df = val_df.copy()

In [ ]:
# Lower text, remove punctuation and stopwords
def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove stopwords
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))

    return text

In [ ]:
train1_df['text'] = train1_df['text'].apply(clean_text)
val1_df['text'] = val1_df['text'].apply(clean_text)

In [ ]:
def get_data(data):
  text = data['text'].to_numpy()
  labels = data['predict'].to_numpy()
  return text, labels

In [ ]:
text, labels = get_data(train1_df)

In [ ]:
# Tokenization
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(text)

In [ ]:
# All texts to same shape
maxlen = 50
def get_sequences(tokenizer, text):
  sequences = tokenizer.texts_to_sequences(text)
  padded_sequences = pad_sequences(sequences, truncating='post', padding='post', maxlen=maxlen)
  return padded_sequences

In [ ]:
padded_train_text = get_sequences(tokenizer, text)

In [ ]:
# Dict of classes
classes = ['ham', 'spam']
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])
train_labels = names_to_ids(labels)

In [ ]:
# Create the model
model = tf.keras.models.Sequential([
tf.keras.layers.Embedding(10000,16,input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(
     loss='sparse_categorical_crossentropy',
     optimizer='adam',
     metrics=['accuracy']
)

In [ ]:
# Train the model
val_text, val_labels = get_data(val1_df)
val_seq = get_sequences(tokenizer, val_text)
val_labels= names_to_ids(val_labels)
h = model.fit(
     padded_train_text, train_labels,
     epochs=20,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)]
)

Epoch 1/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.8403 - loss: 0.8360
Epoch 2/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9625 - loss: 0.1339
Epoch 3/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9886 - loss: 0.0497
Epoch 4/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.9949 - loss: 0.0232
Epoch 5/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9981 - loss: 0.0125
Epoch 6/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9982 - loss: 0.0104
Epoch 7/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.9984 - loss: 0.0110
Epoch 8/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 1.0000 - loss: 0.0016
Epoch 9/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.9988 - loss: 0.0027
Epoch 10/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 1.0000 - loss: 0.0011


In [ ]:
model.evaluate(val_seq, val_labels)

43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9718 - loss: 0.1470


[0.11335518956184387, 0.9755374193191528]

In [ ]:
examples = "sale today! to stop texts call 98912460324"
#examples = clean_text(examples)
examples_token = tokenizer.texts_to_sequences([examples])
paddedSequence = pad_sequences(examples_token, truncating = 'post', padding='post', maxlen=maxlen)
p = model.predict(np.expand_dims(paddedSequence[0], axis=0))[0]
pred_class=index_to_class[np.argmax(p).astype('uint8')]
print('Sentence:', examples)
print('Predicted Emotion: ', pred_class)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Sentence: sale today! to stop texts call 98912460324
Predicted Emotion:  spam


In [ ]:
model.predict(examples)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 384ms/step


array([[9.9870014e-01, 1.0960001e-03, 7.6783224e-05, 3.9722512e-05,
        5.5704051e-05, 3.1598738e-05]], dtype=float32)

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  examples_token = tokenizer.texts_to_sequences([pred_text])
  paddedSequence = pad_sequences(examples_token, truncating = 'post', padding='post', maxlen=maxlen)
  p = model.predict(np.expand_dims(paddedSequence[0], axis=0))[0]
  pred_class=index_to_class[np.argmax(p).astype('uint8')]
  prediction = [p[np.argmax(p)], pred_class]
  return (prediction)

pred_text = "Hello, world."

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step
[0.9994289, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[0.9969798, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[0.9942925, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0.99894816, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0.9980677, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
[0.9980432, 'spam']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
[0.99936813, 'ham']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
[0.99904674, 'ham']
You passed the challenge. Great job!
